# Image classification with CNNs
[![Open in Layer](https://development.layer.co/assets/badge.svg)](https://development.layer.co/layer/cnn) [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/layerai/examples/blob/main/image-classification/image_classification_with_CNNs.ipynb) [![Layer Examples Github](https://badgen.net/badge/icon/github?icon=github&label)](https://github.com/layerai/examples/tree/main/image-classification)

In [ ]:
!pip install --upgrade layer-sdk -qqq

In [ ]:
!layer --version

Layer, version 0.9.341296


In [ ]:
import layer

In [ ]:
# layer.logout()
# layer.login("https://app.layer.ai/")
layer.login("https://development.layer.co/")

Please open the following link in your web browser. Once logged in, copy the code and paste it here.
https://auth.development.layer.co/authorize?response_type=code&code_challenge=j5--kaGMXp_TO8OIAui8C6019NqBkQLsLfwxNb7H430&code_challenge_method=S256&client_id=nv7k9jFnEHhMVQT6gWRoHMYkEtGUedtz&redirect_uri=https://development.layer.co/oauth/code&scope=offline_access&audience=https://development.layer.co
Code: k6bmOxztKqI7j2NqB5Vui9mtzFC0aE_vg12MlaMojLV8I
Successfully logged into https://development.layer.co/


In [ ]:
from layer.decorators import model, resources, pip_requirements, fabric

In [ ]:
layer.init("derrick-cnn")

Project(name='derrick-cnn', raw_datasets=[], derived_datasets=[], featuresets=[], models=[], path=PosixPath('.'), project_files_hash='', readme='', organization=Organization(id=UUID('d7325da3-0646-4fa6-855d-8d19eece8b79'), name='layer'), _id=UUID('54de9921-fdc4-414a-89ca-8ce39094fe8f'), functions=[])

In [ ]:
pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=165a26acdf10ac2913e55b38771a6ecc5f85f017d891b9b227b2268e1086c9a4
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [ ]:
@pip_requirements(packages=["wget","tensorflow","keras"])
@fabric("f-medium")
@model("cnn")
def train():
  from tensorflow.keras.preprocessing.image import ImageDataGenerator
  import tensorflow as tf
  import wget
  from tensorflow import keras
  from tensorflow.keras import Sequential
  from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout
  from tensorflow.keras.preprocessing.image import ImageDataGenerator
  from tensorflow.keras.callbacks import EarlyStopping
  import os
  import matplotlib.pyplot as plt 
  import pandas as pd
  import zipfile
  wget.download("https://namespace.co.ke/ml/train.zip")
  with zipfile.ZipFile('train.zip', 'r') as zip_ref:
    zip_ref.extractall('train')
  base_dir = 'train/train'
  filenames = os.listdir(base_dir)
  categories = []
  for filename in filenames:
      category = filename.split('.')[0]
      if category == 'dog':
          categories.append("dog")
      else:
          categories.append("cat")

  df = pd.DataFrame({'filename': filenames,'category': categories})
  train_datagen = ImageDataGenerator(rescale=1./255, 
                                   shear_range=0.2,
                                   zoom_range=0.2, 
                                   horizontal_flip=True,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   validation_split=0.2
                                   )
  validation_gen = ImageDataGenerator(rescale=1./255,validation_split=0.2)
  image_size = (200, 200)
  training_set = train_datagen.flow_from_dataframe(df,base_dir,
                                                 seed=101,                                                 
                                                 target_size=image_size,
                                                 batch_size=32,
                                                 x_col='filename',
                                                 y_col='category',
                                                 subset = "training",
                                                 class_mode='binary')
  validation_set = validation_gen.flow_from_dataframe(df,base_dir, 
                                               target_size=image_size,
                                               batch_size=32, 
                                               x_col='filename',
                                                 y_col='category',
                                               subset = "validation",
                                               class_mode='binary')
  model = Sequential([
    Conv2D(filters=32,kernel_size=(3,3),  input_shape = (200, 200, 3),activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(filters=32,kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),

    Conv2D(filters=64,kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.25),
    Dense(1, activation='sigmoid')])
  model.compile(optimizer='adam',
              loss=keras.losses.BinaryCrossentropy(),
              metrics=[keras.metrics.BinaryAccuracy()])
  callback = EarlyStopping(monitor='loss', patience=3)
  epochs=1
  history = model.fit(training_set,validation_data=validation_set, epochs=epochs,callbacks=[callback])
  metrics_df = pd.DataFrame(history.history)
  layer.log({"metrics DataFrame": metric_df)
  loss, accuracy = model.evaluate(validation_set)
  print('Accuracy on test dataset:', accuracy)
  metrics_df[["loss","val_loss"]].plot();
  metrics_df[["binary_accuracy","val_binary_accuracy"]].plot();
  return model

In [ ]:
# Run the project on Layer Infra
layer.run([train])

Output()

Run(project_name='derrick-cnn')

In [ ]:
from PIL import Image
import numpy as np
from keras.preprocessing import image

In [ ]:
model = layer.get_model('layer/derrick-cnn/models/cnn').get_train()

In [ ]:
!wget --no-check-certificate \
    https://upload.wikimedia.org/wikipedia/commons/c/c7/Tabby_cat_with_blue_eyes-3336579.jpg \
    -O /tmp/cat.jpg

--2022-03-30 07:39:41--  https://upload.wikimedia.org/wikipedia/commons/c/c7/Tabby_cat_with_blue_eyes-3336579.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 91.198.174.208, 2620:0:862:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|91.198.174.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2114775 (2.0M) [image/jpeg]
Saving to: ‘/tmp/cat.jpg’

/tmp/cat.jpg        100%[===================>]   2.02M  --.-KB/s    in 0.08s   

2022-03-30 07:39:41 (25.9 MB/s) - ‘/tmp/cat.jpg’ saved [2114775/2114775]



In [ ]:
test_image = image.load_img('/tmp/cat.jpg', target_size=(200, 200))
test_image = image.img_to_array(test_image)

In [ ]:
test_image = np.expand_dims(test_image, axis=0)
prediction = model.predict(test_image)


In [ ]:
if prediction[0][0]>0.5:
  print(" is a dog")
else:
   print(" is a cat")

 is a cat
